In [3]:
import numpy as np
from scipy import linalg as LA
import copy

class gaussElim():
    """
    Élimination gaussienne avec pivotement partiel.
     Entrer: A est une matrice numpy n x n
            b est un tableau numpy n x 1
     sortie: x est la solution de Ax = b
             avec les entrées permutées dans
             conformément au pivotement
             fait par l'algorithme
     post-condition: A et b ont été modifiés.
     :return
    """

    def __init__(self, A, b, doPricing=True):

        self.A = A                      # entrée: A est une matrice numpy n x n
        self.b = b                      # b est un tableau numpy n x 1
        self.doPricing = doPricing

        self.n = None                   # n is the length of A
        self.x = None                   # x is the solution of Ax=b

        self._validate_input()          # méthode qui valide l'entrée
        self._elimination()             #méthode qui conduit l'élimination
        self._backsub()                 # méthode qui procède à la substitution inverse

    def _validate_input(self):
        self.n = len(self.A)
        if self.b.size != self.n:
            raise ValueError("Argument non valide: tailles incompatibles entre" +
                             "A & b.", self.b.size, self.n)

    def _elimination(self):
        """
            k représente la ligne pivot actuelle. Puisque GE traverse la matrice dans le
         triangle supérieur droit, nous utilisons également k pour indiquer la k-ième diagonale
         index de colonne.
        :return
        """

        # Elimination
        for k in range(self.n - 1):
            if self.doPricing:
                # Pivot
                maxindex = abs(self.A[k:, k]).argmax() + k
                if self.A[maxindex, k] == 0:
                    raise ValueError("Matrix est singulier.")
                # Swap
                if maxindex != k:
                    self.A[[k, maxindex]] = self.A[[maxindex, k]]
                    self.b[[k, maxindex]] = self.b[[maxindex, k]]
            else:
                if self.A[k, k] == 0:
                    raise ValueError("L'élément pivot est égal à zéro.") #il faut que pricing == True
            # Éliminer
            for row in range(k + 1, self.n):
                multiplier = self.A[row, k] / self.A[k, k]
                self.A[row, k:] = self.A[row, k:] - multiplier * self.A[k, k:]
                self.b[row] = self.b[row] - multiplier * self.b[k]

    def _backsub(self):
        #Substitution arrière
        self.x = np.zeros(self.n)
        for k in range(self.n - 1, -1, -1):
            self.x[k] = (self.b[k] - np.dot(self.A[k, k + 1:], self.x[k + 1:])) / self.A[k, k]


def gaussJordan(A,b):

    n,m = A.shape
    # devrait vérifier nos hypothèses, par ex. n == m
    C = np.zeros((n,m+1),float)
    C[:,0:n],C[:,n] = A, b

    for j in range(n):
        # effectuez un pivotement partiel.
        p = j #l'élément diagonal courant est pivot par défaut
        #rechercher un autre pivot en recherchant le plus grand élément de la colonne
        for i in range(j+1,n):
            if abs(C[i,j]) > abs(C[p,j]): p = i
        if abs(C[p,j]) < 1.0e-16:
            print("matrice est singulière")
            return b 
        # permuter les lignes pour obtenir le plus grand élément de magnitude sur la diagonale (CASSÉ)
        # C[p,:],C[j,:] = copy(C[j,:]),copy(C[p,:])
        # Maintenant, faites la mise à l'échelle et l'élimination.
        pivot = C[j,j]
        C[j,:] = C[j,:] / pivot
        for i in range(n):
            if i == j: continue
            C[i,:] = C[i,:] - C[i,j]*C[j,:]
    I,x = C[:,0:n],C[:,n]
    return x